In [1]:
!pip install git+https://github.com/finetuneanon/transformers@gpt-neo-dungeon-localattention1
!nvidia-smi

Tue Apr 27 16:21:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    44W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from transformers import GPTNeoForCausalLM, AutoConfig
import torch

In [3]:
config = AutoConfig.from_pretrained("EleutherAI/gpt-neo-2.7B")
# extra_layers = 26                 # 58 layers
# extra_heads = 4                   # 24 heads, 3072 hidden
# Parameter count: 6.730748928
# extra_layers = 8                  # 40 layers
# extra_heads = 8                   # 28 heads, 3584 hidden
# Parameter count: 6.354528768
extra_layers = 0                  # 32 layers
extra_heads = 12                  # 32 heads, 4096 hidden, GPT-3 6.7B
# Parameter count: 6.658011136
config.attention_layers.extend(["global", "local"] * (extra_layers // 2))
config.attention_types[0][0][1] = 16 + (extra_layers // 2)
config.num_layers = 32 + extra_layers
config.num_heads = 20 + extra_heads
config.hidden_size = 128 * config.num_heads
model = GPTNeoForCausalLM(config).half().cuda()
print("Parameter count: " + str(sum(p.numel() for p in model.parameters() if p.requires_grad) / 1000000000))
model = model.eval()

Parameter count: 6.658011136


In [9]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated() / 1024. / 1024.)
with torch.no_grad():
  for i in range(10):
    ids = torch.zeros(1,2048).int().cuda()
    output = model(ids)
    del ids
    del output.loss
    del output.logits
    del output
print(torch.cuda.max_memory_allocated() / 1024. / 1024.)
!nvidia-smi | grep MiB

12763.1640625
14687.18798828125
| N/A   64C    P0   196W / 250W |  15973MiB / 16280MiB |    100%      Default |
